In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor


import pickle
import joblib

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression

<h2> Entreno el modelo </h2>

In [3]:
#me traigo el dataset de entrenamiento

diamonds= pd.read_csv('./../data/diamonds_train.csv')

diamonds.head()

,Index_ID,Clarity,Color,Cut,Depth,x,y,z,Table,Price,City,Carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,6.83,6.79,4.25,58.0,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,4.35,4.38,2.75,57.0,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,5.62,5.53,3.65,55.0,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,4.68,4.72,3.00,56.0,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,6.55,6.51,3.95,59.0,4882,Dubai,1.02


In [4]:
# creo una nueva variables 
diamonds['xyz'] = diamonds['x'] * diamonds['y']*diamonds['z']*diamonds['Table']*diamonds['Depth']*diamonds['Carat']
diamonds['T/D'] = diamonds['Table'] / diamonds['Depth']
diamonds['C/C'] = diamonds['Clarity'] + diamonds['Color']



In [783]:
#columnas con las que me quedo
X_train_original= diamonds [['Carat', 'Cut','Color', 'Table','Depth','xyz','T/D','C/C','City', 'Clarity']]
y= diamonds['Price']


In [784]:
# get dummies, escalado y entrenar mi modelo

X_train_dummies= pd.get_dummies(X_train_original)

#scaler=StandardScaler()
#X_train_scaled= scaler.fit_transform(X_train_dummies)

X= X_train_dummies 
y= diamonds['Price']


In [785]:
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<h2> Compruebo mi error </h2>

In [786]:
model= RandomForestRegressor(max_depth=12)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)**0.5

print('rmse RandomForestRegressor', rmse)

rmse RandomForestRegressor 595.2378441438186


<h2> Hago la prediccion </h2>

In [787]:
test_df= pd.read_csv('./data/diamonds_test.csv')
test_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,city
0,0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


<h2> Preparo el dataframe como he hecho con el train </h2>

In [788]:
# creo una nueva columna para quitarme x, y z

test_df['xyz'] = test_df['x'] * test_df['y']*diamonds['z']*test_df['table']*test_df['depth']*test_df['carat']
test_df['T/D'] = test_df['table'] / test_df['depth']
test_df['C/C'] = test_df['clarity'] + test_df['color']
test_df['Carat/City'] =  test_df['city']+ test_df['cut']+ test_df['color']+ test_df['clarity']

#columnas con las que me quedo

test_df_mod= test_df [['carat', 'cut','color', 'table','depth','xyz','T/D','C/C','city','clarity']]


# get dummies y escalado

test_df_mod= pd.get_dummies(test_df_mod)

#scaler=StandardScaler()
#X_test= scaler.fit_transform(test_df_mod)

X_test= test_df_mod

In [790]:
#hago la predicción de los valores de y con mi modelo
y_pred= model.predict(X_test)
len(y_pred)

/Users/Izaskun/miniconda3/envs/machine/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- carat
- city_Amsterdam
- city_Antwerp
- city_Dubai
- city_Kimberly
- ...
Feature names seen at fit time, yet now missing:
- Carat
- City_Amsterdam
- City_Antwerp
- City_Dubai
- City_Kimberly
- ...

  warnings.warn(message, FutureWarning)


13485

In [791]:
# creo el csv en el formato que hay que subirlo
submission = pd.DataFrame({'id': test_df['id'] , 'price': y_pred})
submission.to_csv('./data_submission/submission_4_4_3.csv',index=False)